# Training

## Import data and libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn import set_config
set_config(transform_output="pandas")

url = "https://drive.google.com/file/d/1L23SxwgqjdUeTKikW-L246yOcI12q0_D/view?usp=drive_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
training_data = pd.read_csv(path)

The column 'Id' does not have useful training data, so we do not want it to be part of the data fed into our pipeline.

At the same time, it will be needed in the competition submission. We will therefore use it as the index.

In [ ]:
training_data = training_data.set_index('Id')
training_data

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
5,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,7917,62.0,953,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1457,13175,85.0,1542,3,2,0,2,349,0,0,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1458,9042,66.0,1152,4,2,0,1,0,0,1,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal


## Split data and train model

In [ ]:
# Separate label and split data for training and testing
y = training_data['Expensive'].copy()
X = training_data.drop('Expensive', axis=1).copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# Separte training data into numerical and text
X_num = X_train.select_dtypes(include='number')
X_cat = X_train.select_dtypes(exclude='number')

In [ ]:
# Create pipeline for numerical data
num_pipe = make_pipeline(SimpleImputer(strategy='mean'))

In [ ]:
# Create pipeline for text data
cat_pipe = make_pipeline(SimpleImputer(strategy='constant', fill_value='N_A'),
                         OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=False))

In [ ]:
# Assemble pipelines into preprocessor and full pipeline
preprocessor = make_column_transformer((num_pipe, X_num.columns),
                                       (cat_pipe, X_cat.columns))

kneighbors_pipe = make_pipeline(preprocessor, KNeighborsClassifier())

In [ ]:
kneighbors_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF...
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
       'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('kneighborsclassifier', KNeighborsClassifier())])

## Assess performance and further train model(s)

In [ ]:
accuracy_score(y_true=y_train,
               y_pred=kneighbors_pipe.predict(X_train))

0.9452054794520548

In [ ]:
assessment_df = pd.DataFrame(columns=['train', 'test'], index=['kneighbors_baseline'])
assessment_df.loc['kneighbors_baseline', 'train'] = accuracy_score(y_true=y_train, y_pred=kneighbors_pipe.predict(X_train))
assessment_df.loc['kneighbors_baseline', 'test'] = accuracy_score(y_true=y_test, y_pred=kneighbors_pipe.predict(X_test))
assessment_df

,train,test
kneighbors_baseline,0.945205,0.90411


In [ ]:
# Tune hyperparameters, change up your preprocessor and try other models!
# Use cross-validation to get the most out of your training data!

## Fit one last time on full training data

Once you have your best model chosen, fit it one last time on the whole training data.

In [ ]:
kneighbors_pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF...
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
       'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('kneighborsclassifier', KNeighborsClassifier())])

# Predict on testing data

## Import testing data
Remember, everything done up until now has been performed on your training data. The *real* testing data has no labels (visible to you).

In [ ]:
url = "https://drive.google.com/file/d/15PfmTxmavQCT-f7iY9tgwWxm9t4GRees/view?usp=drive_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
testing_data = pd.read_csv(path)
testing_data = testing_data.set_index('Id')

Use your model's `.predict()` method to create predictions.

In [ ]:
kneighbors_pipe.predict(testing_data)

array([0, 0, 0, ..., 0, 0, 0])

The output of `.predict()` is an array. Save this as a column on `testing_data`.

In [ ]:
testing_data['Expensive'] = kneighbors_pipe.predict(testing_data)

In [ ]:
# Export the column 'Expensive' along with the index to create a submission file
testing_data['Expensive'].to_csv('./submission.csv')

In [ ]:
# Colab only
from google.colab import files
files.download('./submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>